# Sample code for generating dummy data for monitoring

## Install packages

In [ ]:
# %pip install langfuse pandas weaviate-client jupyter openai ragas  # or
%pip install -r requirements.txt

## Setup env vars

1. Create a weaviate cluster at https://console.weaviate.cloud/create-cluster \
   [**Note**: ingested data in free account expires after 14 days]
2. *Wait* until the cluster is created
3. Click on the ![Weaviate code button](weaviate-connect.svg) button to show code to connect to the cluster
4. Add the REST endpoint into `WCS_URL` environment variable \
   `export WCS_URL="https://url.here"`
5. Also set the `WCS_API_KEY` environment variable to the admin API key. Click on the **API keys** button to reveal it. \
   `export WCS_API_KEY="api-key-here"`
6. Create new OpenAI API key at https://platform.openai.com/api-keys \
   `export OPENAI_API_KEY="key-here"`
7. Create new Langfuse API keys at https://cloud.langfuse.com/ (First create a project) \
   `export LANGFUSE_SECRET_KEY="secret-key"` \
   `export LANGFUSE_PUBLIC_KEY="public-key"` \
   `export LANGFUSE_HOST="https://cloud.langfuse.com"`

## Imports

In [ ]:
# builtin packages
import os
import json
from textwrap import dedent
from datetime import datetime, timezone

# 3rd party packages
import weaviate
import pandas as pd
from langfuse.openai import OpenAI
from langfuse import Langfuse
from langfuse.decorators import observe

## Connect to weaviate cluster

In [ ]:
# Set these environment variables
URL = os.getenv("WCS_URL")
APIKEY = os.getenv("WCS_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Connect to a WCS instance
wclient = weaviate.connect_to_wcs(
    cluster_url=URL,
    auth_credentials=weaviate.auth.AuthApiKey(APIKEY),
    headers = {
        "X-OpenAI-Api-Key": OPENAI_API_KEY,
    }
)
lf_client = Langfuse()

## Terms
1. **Collection** \
   A collection is like a table
2. **Field** in a collection \
   A field is like a column of a table
3. **Vector** \
   A vector is an sequence of numbers. [Ref: ChatGPT]
4. **Embedding** \
   An embedding is the mapping of an object to a vector.

## Generating vector embeddings

The following things will happen when you run the code given below:
1. Read the `rag.csv` and do a basic cleanup
2. Create the collection with the specified schema and OpenAI's `text-embedding-3-large` embedding model. Click [here](https://platform.openai.com/docs/guides/embeddings) for more info on other OpenAI's models.
3. Import the records from the CSV to the newly created collection

In [ ]:
def get_or_create_collection(collection_name: str, *, recreate: bool) -> weaviate.collections.collection.Collection:
    if recreate:
        wclient.collections.delete(collection_name)

    try:
        wclient.collections.list_all()[collection_name]
    except KeyError:
        pass
    else:
        return wclient.collections.get(collection_name)

    df = pd \
        .read_csv('rag.csv', dtype=str) \
        .dropna(ignore_index=True) \
        .drop_duplicates(subset=["question"], ignore_index=True) \
        .map(str.strip)
    
    collection = wclient.collections.create(
        name=collection_name,
        vectorizer_config=weaviate.classes.config.Configure.Vectorizer.text2vec_openai(model="text-embedding-3-large"),
        properties=[
            # Define db schema here
            weaviate.classes.config.Property(name="question", data_type=weaviate.classes.config.DataType.TEXT),
            weaviate.classes.config.Property(name="answer", data_type=weaviate.classes.config.DataType.TEXT),
        ],
    )

    # Import the records from the CSV to our weaviate collection
    with collection.batch.rate_limit(requests_per_minute=100) as batch:
        for _, row in df.iterrows():
            batch.add_object(properties=row.to_dict())

    return collection

In [ ]:
collection = get_or_create_collection("Medqna", recreate=False)

## Get top 3 matches

In [ ]:
def get_top_3_matches(message, trace=None):
    _start = datetime.now(timezone.utc)
    results = collection \
        .query \
        .near_text(message, limit=3) \
        .objects
    results_new = [x.properties for x in results]
    context = "\n".join(
        f'Question: "{x['question']}" , Answer: """{x['answer']}"""\n\n'
        for x in results_new
    )
    _end = datetime.now(timezone.utc)
    if trace is not None:
        trace.generation(
            name = "weaviate-near-text",
            input=message,
            output=context,
            metadata={"top_results": [*results_new], "collection_name": collection.name, "limit": 3},
            start_time=_start,
            end_time=_end,
        )
    return context, results_new

In [ ]:
message = "What food to avoid during pregnancy ?."

In [ ]:
context, _ = get_top_3_matches(message)

In [ ]:
prompt = lf_client.get_prompt("medical-doctor")

In [ ]:
openai_client = OpenAI()

response = openai_client.chat.completions.create(
    # model="gpt-4-turbo-preview",
    model="gpt-4o",
    messages=[
        {"role": "system", "content": prompt.compile(context=context)},
        {"role": "user"  , "content": message},
    ],
    temperature=0.2,
    max_tokens=256,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
)

In [ ]:
# import metrics
from ragas.metrics import faithfulness, answer_relevancy, context_precision
from ragas.metrics.critique import harmfulness
 
# metrics you chose
metrics = [faithfulness, answer_relevancy, harmfulness]

In [ ]:
from ragas.run_config import RunConfig
from ragas.metrics.base import MetricWithLLM, MetricWithEmbeddings
 
 
# util function to init Ragas Metrics
def init_ragas_metrics(metrics, llm, embedding):
    for metric in metrics:
        if isinstance(metric, MetricWithLLM):
            metric.llm = llm
        if isinstance(metric, MetricWithEmbeddings):
            metric.embeddings = embedding
        run_config = RunConfig()
        metric.init(run_config)

In [ ]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings
 
# wrappers
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
 
llm = ChatOpenAI()
emb = OpenAIEmbeddings()
 
init_ragas_metrics(
    metrics,
    llm=LangchainLLMWrapper(llm),
    embedding=LangchainEmbeddingsWrapper(emb),
)
@observe()
async def score_with_ragas(query, chunks, answer):
    scores = {}
    for m in metrics:
        print(f"calculating {m.name}")
        scores[m.name] = await m.ascore(
            row={"question": query, "contexts": chunks, "answer": answer}
        )
    return scores

In [ ]:
@observe()
def get_summarized_answer(message, trace):
    context, _ = get_top_3_matches(message, trace)

    openai_client = OpenAI()

    response = openai_client.chat.completions.create(
        # model="gpt-4-turbo-preview",
        model="gpt-4o",
        messages=[
            {"role": "system", "content": prompt.compile(context=context)},
            {"role": "user"  , "content": message},
        ],
        temperature=0.2,
        max_tokens=256,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        trace_id=trace.id,
    )
    output = response.choices[0].message.content

    # Update the trace with the output
    trace.update(output=output)

    return output, context

## Create a trace when you get a message

In [ ]:
trace = lf_client.trace(name = "rag", input=message, start_time=datetime.now(timezone.utc))

## Get output and context

In [ ]:
output, context = get_summarized_answer(message, trace)

## Calculate the scores

In [ ]:
ragas_scores = await score_with_ragas(message, [context], output)

## Send the scores

In [ ]:
for m in metrics:
    trace.score(name=m.name, value=ragas_scores[m.name])

## Mark the end of the trace

In [ ]:
trace.update(end_time=datetime.now(timezone.utc))